#### This script uses LocalColabFold to repair the PDB files in AbAGym, specifically:
* Add missing side-chain atoms
* Add missing residues
* Mutate some wrong residues in the original files to the correct ones  

In [54]:
import os
import warnings
from pathlib import Path

import natsort
import pandas as pd
import requests
from Bio import SeqIO
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq
from Bio.PDB import PDBParser
from Bio.SeqUtils import seq3, seq1
from pdbfixer import PDBFixer

warnings.filterwarnings("ignore")

working_dir = Path(os.getcwd()).parent
rcsb_dir = working_dir / "RCSB_PDBs"
repair_dir = working_dir / "Repaired_PDBs"
colabfold_dir = working_dir / "ColabFold"

##### Produce a table of metadata

In [45]:
metatbl = pd.DataFrame(columns=["project", "dms_table", "dms_pdb", "rcsb_id"])

for data_dir in working_dir.iterdir():
    if data_dir.name.count("_") == 2 and not data_dir.name.count("."):  # Excludes dirs like "src"
        if (data_dir/"PDB_structures").exists() and (data_dir/"DMS_interface_data").exists():
            # for fpath in (data_dir/"DMS_interface_data").iterdir():
                # project_id = "_".join(fpath.name.split("_")[:-3])
            for fpath in (data_dir/"PDB_structures").iterdir():
                project_id = ".".join(fpath.name.split(".")[:-1])
                rcsb_id = project_id.split("_")[-1]
                metatbl.loc[len(metatbl.index)] = [
                    project_id, 
                    str(fpath), 
                    str(data_dir / f"PDB_structures/{project_id}.pdb"), 
                    rcsb_id
                ]

metatbl.to_csv(working_dir / "DMS_metadata.csv", index=False)

##### Crawl RCSB PDB
For samples for which the PDB file is unavailable in RCSB, PDB file from other source like SAbDab is used.

Normlly, disbale this block as this is only required to be run at the first time.

In [ ]:
# for i in metatbl.index:
#     rcsb_id = metatbl.loc[i, "rcsb_id"]
#     resp = requests.get(f"https://files.rcsb.org/view/{rcsb_id}.pdb")
#     if resp.status_code == 200:
#         with open(f"{rcsb_dir}/{rcsb_id}.pdb", "w") as fp:
#             fp.write(resp.content.decode())
#     else:
#         print(f"{rcsb_id}: Unable to crawl the PDB file")

##### Check PDB files
Get the mapping from PDB ATOM sequence to PDB SEQRES sequence 

In [46]:
def get_seqres_atom_seq_mapper(rcsb_file):
    chain_to_seqres_seq = {r.id[-1]: str(r.seq) for r in SeqIO.parse(rcsb_file, "pdb-seqres")}
    chain_to_atom_seq = {r.id[-1]: str(r.seq) for r in SeqIO.parse(rcsb_file, "pdb-atom")}
    atom_to_seqres = {}
    seqres_to_chain_id = {}
    for chain in chain_to_seqres_seq:
        atom_to_seqres[chain_to_atom_seq[chain]] = chain_to_seqres_seq[chain]
        seqres_to_chain_id[chain_to_seqres_seq[chain]] = chain
    return atom_to_seqres, seqres_to_chain_id


# def get_seqres_lines(chain, seq):
#     seq = [seq3(a).upper() for a in list(seq)]
#     seq_len = len(seq)
#     seqres_lines = []
#     i = 0
#     for i in range(seq_len//13):
#         seq_perline = seq[13*i:13*i+13]
#         line = f"SEQRES{str(i+1):^5}{chain}{str(seq_len):^7}{' '.join(seq_perline)}"
#         seqres_lines.append(line)
#     seq_perline = seq[13*i+13:]
#     line = f"SEQRES{str(i+2):^4} {chain}{str(seq_len):^7}{' '.join(seq_perline)}"
#     seqres_lines.append(line)
#     seqres = "\n".join(seqres_lines)
#     return seqres


# def write_seqres(sample_file, rcsb_file, outdir):
#     chain_to_atom_seq = {
#         r.id[-1]: str(r.seq) 
#         for r in list(SeqIO.parse(sample_file, "pdb-atom"))
#     }
#     rcsb_atom_to_seqres = get_seqres_atom_seq_mapper(rcsb_file)
#     chain_to_seqres_seq = {
#         chain: rcsb_atom_to_seqres[seq]
#         for chain, seq in chain_to_atom_seq.items()
#     }
#     seqres_lines = []
#     for chain, seq in chain_to_seqres_seq.items():
#         seqres_lines.append(get_seqres_lines(chain, seq))
#     seqres_lines = "\n".join(seqres_lines)
#     # Merge structure and seqres
#     with open(sample_file, "r") as fp:
#         pdb_content = fp.read()
#     outfile = f"{outdir}/{Path(sample_file).name}"
#     with open(outfile, "w") as fp:
#         fp.write(seqres_lines+"\n"+pdb_content)
#     return outfile


def get_dms_rcsb_chain_id_map(dms_pdb_file, rcsb_pdb_file):
    chain_to_atom_seq = {
        r.id[-1]: str(r.seq) 
        for r in list(SeqIO.parse(dms_pdb_file, "pdb-atom"))
    }
    rcsb_atom_to_seqres, seqres_to_chain_id = get_seqres_atom_seq_mapper(rcsb_pdb_file)
    return {
        chain_id: seqres_to_chain_id[rcsb_atom_to_seqres[chain_to_atom_seq[chain_id]]]
        for chain_id in chain_to_atom_seq
    }


for i in metatbl.index:
    try:
        id_map = get_dms_rcsb_chain_id_map(
            metatbl.loc[i, "dms_pdb"],
            f"{rcsb_dir}/{metatbl.loc[i, 'rcsb_id']}.pdb"
        )
        print(id_map)
    except Exception as e:
        print(f"{metatbl.loc[i, 'project']}: {str(e)}")
    metatbl.loc[i, "id_map"] = str(id_map)

metatbl.to_csv(working_dir / "DMS_metadata.csv", index=False)

{'H': 'H', 'L': 'L', 'V': 'V'}
{'A': 'A', 'H': 'H', 'L': 'L'}
{'A': 'G', 'H': 'N', 'L': 'S'}
{'A': 'A', 'H': 'H', 'L': 'L'}
{'A': 'A', 'H': 'C', 'L': 'B'}
{'A': 'E', 'H': 'H', 'L': 'L'}
{'A': 'A', 'H': 'H', 'L': 'L'}
{'A': 'Z', 'H': 'X', 'L': 'Y'}
{'A': 'C', 'H': 'M', 'L': 'N'}
{'A': 'R', 'H': 'H', 'L': 'L'}
{'A': 'R', 'H': 'H', 'L': 'L'}
{'A': 'E', 'H': 'A', 'L': 'B'}
{'A': 'R', 'H': 'H', 'L': 'L'}
{'A': 'R', 'H': 'H', 'L': 'L'}
{'A': 'C', 'H': 'E', 'L': 'G'}
{'A': 'A', 'H': 'H', 'L': 'L'}
{'A': 'A', 'H': 'H', 'L': 'L'}
{'A': 'E', 'H': 'H', 'L': 'L'}
CV30_6xe1: 'IVLTQSPGTLSLSPGERATLSCRASQSVSSSYLAWYQQKPGQAPRLLIYGASSRATGIPDRFSGSGSGTDFTLTISRLEPEDFAVYYCQQYGSSPQTFGQGTKLEIKRTVAAPSVFIFPPSDEQLKSGTASVVCLLNNFYPREAKVQWKVDNALQSGNSQESVTEQDSKDSTYSLSSTLTLSKADYEKHKVYACEVTHQGLSSPVTKSFNR'
{'A': 'E', 'H': 'H', 'L': 'L'}
{'A': 'A', 'H': 'C', 'L': 'B'}
{'A': 'C', 'H': 'A', 'L': 'B'}
{'A': 'R', 'H': 'A', 'L': 'B'}
{'A': 'C', 'H': 'A', 'L': 'B'}
{'A': 'E', 'H': 'H', 'L': 'L'}
{'A': 'R', 'H': 'H', 'L': 'L'}


In [73]:
pdbs_to_mutate = {
    "BD55-3152_7wr8": [("A", 339, 'D', 'G')],
    "BD55-5840_7wrz": [("A", 339, 'D', 'G')],
    "CAB-A17_8c2r": [("A", 417, 'N', 'K'), ("A", 496, 'S', 'G'), ("A", 477, 'N', 'S'), ("A", 501, 'Y', 'N'), ("A", 505, 'H', 'Y'), ("A", 493, 'R', 'Q')],
    "D441_1mlc": [("H", 32, 'Y', 'N'), ("H", 31, 'T', 'N'), ("H", 50, 'E', 'Y'), ("H", 33, 'W', 'L')],
    "EDE1-C8_5lbs": [("A", 317, 'I', 'V')],
    "ZKA64_5kvf": [("A", 343, 'A', 'V')],
    "ZV-67_5kvg": [("A", 393, 'E', 'D'), ("A", 317, 'I', 'V')]
}


def check_seq_entirety(pdbfixer: PDBFixer):
    missing_resids = pdbfixer.missingResidues
    chains = list(pdbfixer.topology.chains())
    flags = []
    for chain_idx, posit in missing_resids.keys():
        chain = chains[chain_idx]
        if posit == 0 or len(list(chain.residues())) == posit:
            flags.append(1)
        else:
            flags.append(2)
    if 2 in flags:
        return 2
    elif 1 in flags:
        return 1
    else:
        return 0


for i in metatbl.index:
    rcsb_id = metatbl.loc[i, "rcsb_id"]
    rcsb_pdb_file = f"/home/dongli/DMS_antibody_dataset/RCSB_PDBs/{rcsb_id}.pdb"
    fixer = PDBFixer(rcsb_pdb_file)
    fixer.findMissingResidues()
    fixer.findMissingAtoms()
    missing_atoms = fixer.missingAtoms
    # print(metatbl.loc[i, "project"])
    seq_entirety_flag = check_seq_entirety(fixer)
    metatbl.loc[i, "seq_entirety"] = seq_entirety_flag
    metatbl.loc[i, "atom_entirety"] = int(bool(missing_atoms))
    mutations = pdbs_to_mutate.get(metatbl.loc[i, "project"], [])
    chain_to_muts = {}
    for mut in mutations:
        try:
            chain_to_muts[mut[0]].append(mut[1:])
        except KeyError:
            chain_to_muts[mut[0]] = [mut[1:]]
    metatbl.loc[i, "mutation"] = str(chain_to_muts)

metatbl = metatbl.astype({"seq_entirety": int, "atom_entirety": int})
metatbl.to_csv(working_dir / "DMS_metadata.csv", index=False)

##### Get target sequences

In [75]:
for i in metatbl.index:
    project_id = metatbl.loc[i, "project"]
    chain_id_map = eval(metatbl.loc[i, "id_map"])
    seq_entirety = metatbl.loc[i, "seq_entirety"]
    atom_entirety = metatbl.loc[i, "atom_entirety"]
    chain_to_muts = eval(metatbl.loc[i, "mutation"])
    if seq_entirety == 2 or atom_entirety or mutations:  # Needs a re-modelling
        rcsb_id = metatbl.loc[i, "rcsb_id"]
        print(project_id)
        rcsb_pdb_file = f"/home/dongli/DMS_antibody_dataset/RCSB_PDBs/{rcsb_id}.pdb"
        pdb_info_df = pd.DataFrame(columns=["Chain", "Resi_ID", "Insertion_code", "Resi"])
        struct = PDBParser().get_structure(project_id, rcsb_pdb_file)
        model = struct.child_list[0]
        header = struct.header
        chain_to_seqres = {
            seqrec.id[-1]: str(seqrec.seq).replace("X", "")
            for seqrec in SeqIO.parse(rcsb_pdb_file, "pdb-seqres")
        }
        colabfold_seqs = []
        dfs = []
        for chain_id in chain_id_map:
            rcsb_chain_id = chain_id_map[chain_id]
            pdb_seqres_seq = chain_to_seqres[rcsb_chain_id]
            missing_residue_info = [
                (chain_id, f"{dd['ssseq']}{str(dd['insertion'] or '')}", seq1(dd["res_name"]), 0)
                for dd in header["missing_residues"]
                if dd["chain"] == rcsb_chain_id
            ]
            atom_residue_info = {
                (chain_id, f"{resi.id[1]}{str(resi.id[2])}".replace(" ", ""), seq1(resi.resname), 1) 
                for resi in model.child_dict[rcsb_chain_id].child_list 
                if resi.id[0] == " "
            }
            df = pd.DataFrame(
                [*missing_residue_info, *atom_residue_info],
                columns = ["chain_id", "residue_id", "residue_name", "has_coords"]
            )
            df.sort_values(
                by = ["residue_id"],
                key = lambda x: natsort.natsort_key(x), 
                inplace = True
            )
            dfs.append(df)
            # print(chain_id_map)
            # print(chain_to_muts)
            for mut in chain_to_muts.get(chain_id, []):
                mut_posit = df[df["residue_id"]==str(mut[0])].index[0]
                if df.loc[mut_posit, "residue_name"] != mut[1]:
                    print("Residue type incorrect")
                    continue
                else:
                    df.loc[mut_posit, "residue_name"] = mut[2]
            colabfold_seqs.append("".join(df["residue_name"]))
        SeqIO.write(
            SeqRecord(Seq(":".join(colabfold_seqs)), rcsb_id, description=project_id+":"+"".join(chain_id_map.keys())),
            colabfold_dir / f"fasta/{project_id}.fasta",
            "fasta"
        )
        pd.concat(dfs).to_csv(colabfold_dir / f"data/{project_id}.csv", index=False)

4edw
COVA2-04_7jmo
C119_7k8w
CC12.3_6xc4
BG1-24_7m6i
COVOX-222_7or9
BD55-5840_7wrz
DXP-604_7ch4
C121_7k8x
B38_7xil
P2B-2F6_8dcc
C1A-B12_7kfv
LY-CoV481_7kmi
BD55-3152_7wr8
LY-CoV488_7kmh
COVOX-150_7bei
WIBP-2B11_7e5y
BD55-1239_7wrl
LY-CoV1404_7mmo
LY-CoV555_7kmg
C105_6xcm
REGN10933_6xdg
C135_7k8z
C002_7k8s
C110_7k8v
C121_7k8x
REGN10987_6xdg
C144_7k90
VRC01_5fyk
PG9_3u4e
3BNC117_5v8m
3BN-1074_5t3z
PGT145_5v8l
PGT121_5fyl
AZD1061_7l7e
AZD8895_7l7d
FI6V_3ztn
C179_4hlz
S139_4gms
S2X259_7m7w
S309_7r6w
S2D106_7r7n
S2H13_7jv6
S2H97_7m7w
S304_7jx3
S2H14_7jx3
S2E12_7r6x
S2X35_7r6w
106E6_6n1w
OPV20_6osy
DF1W314_6mph
17D4_6n1v
110D12_6mph
OPV12_6ot1
SIgN-3C_7bua
MZ4_6niu
EDE1-C8_5lbs
EDE1-C10_5h37
118_6udj
FP20-01_6cde
FP16-02_6cdi
VRC34-01_5i8h
FI6V3-H1_3ztn
CR9114_4fqy
FI6V3-H3_3ztj
CAB-A17_8c2r
Cetuximab_1yy9
